Código para pasar de matrices de rotación a orientación en omegas

***Crear nuevo archivo con omegas como orientación***

In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

# Definir una función para leer un archivo pkl
def leer_archivo_pkl(nombre_archivo):
    try:
        with open(nombre_archivo, 'rb') as archivo:
            contenido = pickle.load(archivo)
            return contenido
    except FileNotFoundError:
        print(f"El archivo {nombre_archivo} no se encontró.")
    except Exception as e:
        print(f"Error al leer el archivo {nombre_archivo}: {e}")

# Ruta del archivo pkl
nombre_archivo_pkl = './Pruebas AMASS/Walking_3_poses_pkl_version_synthetic.pkl'
# Llama a la función para leer el archivo pkl
datos = leer_archivo_pkl(nombre_archivo_pkl)

# Extraer matrices de rotación y aceleraciones del diccionario de datos
matrices_rotacion = datos['ori']
aceleraciones = datos['acc']
gt = datos['gt']  # Extraer el diccionario 'gt'


# Diferencial de tiempo
DeltaT = 1/120    

# Listas para almacenar las velocidades angulares y aceleraciones en cada eje
omega_x = []
omega_y = []
omega_z = []
aceleracion_x = []
aceleracion_y = []
aceleracion_z = []



print(aceleraciones[0][0])

# Iterar sobre los valores de j (suponiendo que son 8 en total)
for j in range(8):
    # Iterar sobre todas las matrices de rotación excepto la última
    for i in range(len(matrices_rotacion)-1):
        # Obtener matrices de rotación en los tiempos t0 y t1
        R_t0 = matrices_rotacion[i][j]
        R_t1 = matrices_rotacion[i+1][j]
        
        # Imprimir información de la matriz actual
        # print("Matriz " + str(i) + " ")
        
        # Calcular la derivada temporal de la matriz de rotación
        R_dot = (R_t1 - R_t0) / DeltaT
        
        # Calcular la transpuesta de la derivada temporal de la matriz de rotación
        R_transpose_dot = np.transpose(R_dot)
        
        # Calcular la matriz resultante
        R_transpose_dot_R = np.dot(R_transpose_dot, R_t0)
        
        # Inicializar las listas omega_x, omega_y, omega_z, aceleracion_x, aceleracion_y y aceleracion_z si es necesario
        if len(omega_x) <= j:
            omega_x.append([])
            omega_y.append([])
            omega_z.append([])
            aceleracion_x.append([])
            aceleracion_y.append([])
            aceleracion_z.append([])
        
        # Extraer las componentes de la matriz resultante que representan las velocidades angulares
        omega_x[j].append(R_transpose_dot_R[2, 1] - R_transpose_dot_R[1, 2])
        omega_y[j].append(R_transpose_dot_R[0, 2] - R_transpose_dot_R[2, 0])
        omega_z[j].append(R_transpose_dot_R[1, 0] - R_transpose_dot_R[0, 1])
                        
        # Agregar la aceleración media a las listas de aceleraciones
        aceleracion_x[j].append(aceleraciones[i][j][0])
        aceleracion_y[j].append(aceleraciones[i][j][1])
        aceleracion_z[j].append(aceleraciones[i][j][2])

# Lista de valores de j
valores_j = range(len(omega_x))
print(omega_x)

# Crear un nuevo diccionario para almacenar los datos
nuevos_datos = {'ori': [], 'acc': [], 'gt': gt}  # Agregar el diccionario 'gt' al nuevo diccionario


# Iterar sobre los valores de tiempo
for i in range(len(omega_x[0])):
    instantes_tiempo = []
    # Iterar sobre cada IMU
    for j in range(len(omega_x)):
        # Crear una lista con los valores de omega_x, omega_y y omega_z para cada IMU
        valores_omega = [omega_x[j][i], omega_y[j][i], omega_z[j][i]]
        instantes_tiempo.append(valores_omega)
    # Convertir la lista de instantes de tiempo en un array numpy y añadirlo al diccionario de orientación
    nuevos_datos['ori'].append(np.array(instantes_tiempo))

# Agregar los datos de aceleración al diccionario
nuevos_datos['acc'] = aceleraciones

# Ruta del archivo pkl nuevo
nombre_archivo_pkl_nuevo = './Pruebas AMASS/Walking_3_poses_pkl_version_synthetic_omegas.pkl'

# Guardar los datos en un archivo pkl nuevo
with open(nombre_archivo_pkl_nuevo, 'wb') as archivo_nuevo:
    pickle.dump(nuevos_datos, archivo_nuevo)

print("Se ha creado el archivo pkl nuevo satisfactoriamente.")


[-3.23483032  4.51148852  1.65480868]
[[13.17203302856614, 12.26722373289575, 15.045607847189613, 11.879588321652705, 10.990266350538024, 10.818053088137226, 13.408772873452872, 10.640789363539422, 9.249755701349159, 5.050372036106761, -1.5676347057538496, -5.91905641428532, -4.810164320633134, -8.618214332294869, -8.733659061481811, -5.034065423347996, -6.468731721337329, -6.100974432605568, -3.365205622498264, -2.613502242267768, 0.3049671589687216, -1.6938153558643796, 0.2700065734626172, 0.014907611448884936, -0.08731319506259469, 0.4125840138706157, -0.1621772103476178, -0.4457861920137896, -0.40727863220549376, -1.0886502439856096, -0.3532261277551295, 1.068958243951883, -1.1227994324931831, -0.9710435487618307, -1.3809937729544566, -0.6129002444860521, -1.62295339292496, -1.8740603312347512, -2.1255301746120976, -0.28850420334942417, -2.4460268324630343, -1.1471537935410294, -1.9654671205764092, -3.544207250692221, -3.033547366917522, -2.887132052489206, -4.390216179126702, -5.1

***Leer archivo***

In [4]:
import pickle

def leer_archivo_pkl(nombre_archivo):
    try:
        with open(nombre_archivo, 'rb') as archivo:
            contenido = pickle.load(archivo)
            return contenido
    except FileNotFoundError:
        print(f"El archivo {nombre_archivo} no se encontró.")
    except Exception as e:
        print(f"Error al leer el archivo {nombre_archivo}: {e}")

# Reemplaza 'nombre_del_archivo.pkl' con el nombre de tu archivo pkl
nombre_archivo_pkl = './Walking_3_poses_pkl_version_synthetic.pkl'

# Llama a la función para leer el archivo pkl
contenido_del_archivo = leer_archivo_pkl(nombre_archivo_pkl)

# Imprime el contenido del archivo
if contenido_del_archivo:
    print("Contenido del archivo pkl:")
    print(contenido_del_archivo)


Contenido del archivo pkl:
{'ori': [array([[ 0.01222904, -0.00912191,  0.01600523],
       [-0.00948412,  0.00475801, -0.03614125],
       [ 0.00448355,  0.02433123, -0.00197931],
       [ 0.00810747, -0.02903029,  0.00322395],
       [ 0.01163753, -0.00557311,  0.00652372],
       [ 0.00539696,  0.00066813, -0.00156273],
       [ 0.02545602,  0.02097588, -0.02420156],
       [ 0.00473231, -0.00974074,  0.02655672]]), array([[ 2.41265683e-03, -2.49508126e-03,  4.55378838e-03],
       [-2.34320254e-03,  9.74476686e-04, -9.81152710e-03],
       [ 1.05368210e-03,  5.11514423e-03, -2.66203535e-04],
       [ 2.30652881e-03, -7.95533299e-03,  8.79173755e-04],
       [ 3.61411485e-03, -2.29544268e-03,  1.60639894e-03],
       [ 1.39973109e-03, -8.36541508e-04, -2.14106028e-05],
       [ 5.21992498e-03,  3.97375112e-03, -6.08431752e-03],
       [ 5.66886682e-04, -3.63082341e-03,  6.18781040e-03]]), array([[ 5.29702374e-04, -6.88019582e-04,  1.22729000e-03],
       [-3.11305692e-04,  2.82700770

Representación

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

# Definir una función para leer un archivo pkl
def leer_archivo_pkl(nombre_archivo):
    try:
        with open(nombre_archivo, 'rb') as archivo:
            contenido = pickle.load(archivo)
            return contenido
    except FileNotFoundError:
        print(f"El archivo {nombre_archivo} no se encontró.")
    except Exception as e:
        print(f"Error al leer el archivo {nombre_archivo}: {e}")

# Ruta del archivo pkl
nombre_archivo_pkl = './C2 - Run_pkl_version_synthetic.pkl'

# Llama a la función para leer el archivo pkl
datos = leer_archivo_pkl(nombre_archivo_pkl)

# Extraer matrices de rotación y aceleraciones del diccionario de datos
matrices_rotacion = datos['ori']
aceleraciones = datos['acc']

# Diferencial de tiempo
DeltaT = 1/120    

# Listas para almacenar las velocidades angulares y aceleraciones en cada eje
omega_x = []
omega_y = []
omega_z = []
aceleracion_x = []
aceleracion_y = []
aceleracion_z = []

print(aceleraciones[0][0])

# Iterar sobre los valores de j (suponiendo que son 8 en total)
for j in range(8):
    # Iterar sobre todas las matrices de rotación excepto la última
    for i in range(len(matrices_rotacion)-1):
        # Obtener matrices de rotación en los tiempos t0 y t1
        R_t0 = matrices_rotacion[i][j]
        R_t1 = matrices_rotacion[i+1][j]
        
        # Imprimir información de la matriz actual
        # print("Matriz " + str(i) + " ")
        
        # Calcular la derivada temporal de la matriz de rotación
        R_dot = (R_t1 - R_t0) / DeltaT
        
        # Calcular la transpuesta de la derivada temporal de la matriz de rotación
        R_transpose_dot = np.transpose(R_dot)
        
        # Calcular la matriz resultante
        R_transpose_dot_R = np.dot(R_transpose_dot, R_t0)
        
        # Inicializar las listas omega_x, omega_y, omega_z, aceleracion_x, aceleracion_y y aceleracion_z si es necesario
        if len(omega_x) <= j:
            omega_x.append([])
            omega_y.append([])
            omega_z.append([])
            aceleracion_x.append([])
            aceleracion_y.append([])
            aceleracion_z.append([])
        
        # Extraer las componentes de la matriz resultante que representan las velocidades angulares
        omega_x[j].append(R_transpose_dot_R[2, 1] - R_transpose_dot_R[1, 2])
        omega_y[j].append(R_transpose_dot_R[0, 2] - R_transpose_dot_R[2, 0])
        omega_z[j].append(R_transpose_dot_R[1, 0] - R_transpose_dot_R[0, 1])
                        
        # Agregar la aceleración media a las listas de aceleraciones
        aceleracion_x[j].append(aceleraciones[i][j][0])
        aceleracion_y[j].append(aceleraciones[i][j][1])
        aceleracion_z[j].append(aceleraciones[i][j][2])

# Lista de valores de j
valores_j = range(len(omega_x))
print(valores_j)

# Número de subgráficos (uno para cada valor de j)
num_subplots = len(valores_j)
print(num_subplots)

# Crear subgráficos para velocidades angulares
fig, axs = plt.subplots(num_subplots, 2, figsize=(12, 8), sharex=True)

# Iterar sobre cada valor de j y graficar en el subgráfico correspondiente
for j, ax in zip(valores_j, axs):
    # Plot para velocidades angulares
    ax[0].plot(omega_x[j], label='omega_x')
    ax[0].plot(omega_y[j], label='omega_y')
    ax[0].plot(omega_z[j], label='omega_z')
    ax[0].set_title('Valores de j = {}'.format(j))
    if j == 0:
        ax[0].legend()
    
    # Plot para aceleraciones
    ax[1].plot(aceleracion_x[j], label='aceleracion_x')
    ax[1].plot(aceleracion_y[j], label='aceleracion_y')
    ax[1].plot(aceleracion_z[j], label='aceleracion_z')
    if j == 0:
        ax[1].legend()

# Etiqueta de eje x común para todos los subgráficos
axs[-1, 0].set_xlabel('Índice de tiempo')
axs[-1, 1].set_xlabel('Índice de tiempo')

# Ajustar espaciado entre subgráficos
plt.tight_layout()

# Mostrar la gráfica
plt.show()